In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Third Party
from datasets import load_dataset
from openai import OpenAI
from rich import print
from rich.panel import Panel
from sklearn.metrics import classification_report

# First Party
from sdg_hub import Flow, FlowMetadata, BlockRegistry

import nest_asyncio
nest_asyncio.apply()

/home/lab/esivaram/sdg_hub/venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Classifying news articles


In this tutorial, you’ll learn how to create your own custom data generation flow using SDG Hub. This notebook will walk you through all the essential pieces to make your own flow using `sdg_hub` for any use-case using the fundamental components of sdg_hub: `Blocks` and `Flows`

As an example use-case, we will pick news classification. Classification is a fundamental task in machine learning, where the goal is to assign predefined categories to input data. To address the classic machine learning use-case of news or text classification, we will use sdg_hub and leverage a language model to **classify news articles** with topic labels — specifically using the [AG News dataset](https://huggingface.co/datasets/fancyzhx/ag_news) from Hugging Face.

We’ll go step by step through a progressively improving flow. Each stage builds on the previous one, giving you a practical sense of how you can evolve your flow from using simple heuristics to highly customized and reliable data generation, using different inference paradigms such as self assessment.

### 🔍 Understand the Task
Before we write any prompts or code, we’ll take time to understand what we want the model to do. For this exercise, the task is **text classification** — assigning one of 4 possible categories (e.g., "World", "Sports", "Sci/Tech", "Business") to a given news article

### 🛠️ Build a Basic Annotation Flow and learn the `sdg_hub` way
We’ll start by creating a minimal flow that simply prompts the model to generate topic labels on the unlabeled data. This will use default prompts, simply populating the prompt with the text and asking the model to generate one of the 4 possible labels, with no examples.

### 🎯 Improve with Assessment and Iteration
Next, we’ll refine the flow by adding an assessment step. Iterations and self verification on a task often lead to better performance

Let’s get started by loading a sample of the dataset

In [3]:
dataset = load_dataset("fancyzhx/ag_news")

train_data = dataset["train"].shuffle(seed=42).select(range(500))
test_data = dataset["test"].shuffle(seed=42).select(range(100))

# map the labels to the category names
label_map = train_data.features['label'].names

train_data = train_data.map(lambda x: {"category": label_map[x["label"]]})
test_data = test_data.map(lambda x: {"category": label_map[x["label"]]})

In [4]:
# Group examples by category
examples_by_category = {}
for item in train_data:
    category = item['category']
    if category not in examples_by_category:
        examples_by_category[category] = []
    examples_by_category[category].append(item['text'])

# Print one example from each category in a panel
for category, examples in examples_by_category.items():
    print(Panel(examples[0], title=f"Category: {category}", expand=False))


╭──────────────────────────────────────────────── Category: World ────────────────────────────────────────────────╮
│ Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a      │
│ halt, the day after 18 people died in explosions at a political rally.                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Category: Sports ────────────────────────────────────────────────╮
│ Desiring Stability Redskins coach Joe Gibbs expects few major personnel changes in the offseason and wants to   │
│ instill a culture of stability in Washington.                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Category: Sci/Tech ───────────────────────────────────────────────╮
│ U2 pitches for Apple New iTunes ads airing during baseball games Tuesday will feature the advertising-shy Irish │
│ rockers.                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Category: Business ───────────────────────────────────────────────╮
│ Economy builds steam in KC Fed district The economy continued to strengthen in September and early October in   │
│ the Great Plains and Rocky Mountain regions covered by the Tenth Federal Reserve District, the Federal Reserve  │
│ Bank of Kansas City said Wednesday.                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Simple Data Annotation Pipeline

In this section, we’ll create our **first working flow** to perform classification using a language model. The goal is to understand the building blocks of `sdg_hub` and how we can employ them to get a language model to classify a given text.

### Recap: How `sdg_hub` Works

```mermaid
flowchart LR
    A[Flow] --> B[Blocks] --> C[Prompts]
    C --> D[Generated Data]
```

### Check if our LLM server is working

In [5]:
openai_api_key = "EMPTY" # replace with your inference server api key
openai_api_base = "http://0.0.0.0:8000/v1" # replace with your inference server endpoint


client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

models = client.models.list()
teacher_model = models.data[0].id

# Test the connection with a simple completion
response = client.chat.completions.create(
    model=teacher_model,
    messages=[{"role": "user", "content": "Hello!"}],
    temperature=0.0,
    max_tokens=10
)
completion = response.choices[0].message.content

print(f"Connection successful! {teacher_model}: {completion}")

Connection successful! meta-llama/Llama-3.3-70B-Instruct: Hello. How can I help you today?

# Building a Simple Classification Flow

### Discover Blocks for us to use



In [6]:
BlockRegistry.discover_blocks()

                                                 Available Blocks                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Block Name                ┃ Category   ┃ Description                                                            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ CombineColumnsBlock       │ deprecated │ DEPRECATED: Use TextConcatBlock instead. Combines multiple columns     │
│                           │            │ into a single column using a separator                                 │
│ DuplicateColumns          │ deprecated │ DEPRECATED: Use DuplicateColumnsBlock instead. Duplicates existing     │
│                           │            │ columns with new names according to a mapping dictionary               │
│ FilterByValueBlock        │ deprecated │ DEPRECATED: Use ColumnValueFilterBlock instead. Filters datasets based │
│                           │            │ on column values using various comparison operations                   │
│ FlattenColumnsBlock       │ deprecated │ DEPRECATED: Use MeltColumnsBlock instead. Transforms wide dataset      │
│                           │            │ format into long format by melting columns into rows                   │
│ LLMBlock                  │ deprecated │ DEPRECATED: Use the new modular approach with PromptBuilderBlock,      │
│                           │            │ LLMChatBlock, and TextParserBlock instead                              │
│ RenameColumns             │ deprecated │ DEPRECATED: Use RenameColumnsBlock instead. Renames columns in a       │
│                           │            │ dataset according to a mapping dictionary                              │
│ SamplePopulatorBlock      │ deprecated │ DEPRECATED: Use a router block instead. Populates dataset with data    │
│                           │            │ from configuration files                                               │
│ SelectorBlock             │ deprecated │ DEPRECATED: Use IndexBasedMapperBlock instead. Selects and maps values │
│                           │            │ from one column to another                                             │
│ SetToMajorityValue        │ deprecated │ DEPRECATED: Use UniformColumnValueSetter with                          │
│                           │            │ reduction_strategy='mode' instead. Sets all values in a column to the  │
│                           │            │ most frequent value                                                    │
│ EvaluateFaithfulnessBlock │ evaluation │ Composite block for faithfulness evaluation of question-answer pairs   │
│ EvaluateRelevancyBlock    │ evaluation │ Composite block for relevancy evaluation of question-answer pairs      │
│ VerifyQuestionBlock       │ evaluation │ Composite block for question verification and quality assessment       │
│ ColumnValueFilterBlock    │ filtering  │ Filters datasets based on column values using various comparison       │
│                           │            │ operations                                                             │
│ LLMChatBlock              │ llm        │ Unified LLM chat block supporting 100+ providers via LiteLLM           │
│ PromptBuilderBlock        │ llm        │ Formats prompts into structured chat messages or plain text using      │
│                           │            │ Jinja templates                                                        │
│ TextParserBlock           │ llm        │ Parses and post-processes LLM outputs using tags or regex patterns     │
│ DuplicateColumnsBlock     │ transform  │ Duplicates existing columns with new names according to a mapping      │
│                           │            │ specification                                                          │
│ IndexBasedMapperBlock     │ transform  │ Maps values f

Summary: 22 blocks across 5 categories

It seems all the functionality we are interested in, such as building a prompt, chatting with an llm and parsing its output are under the `llm` category in sdg_hub. Lets start there.

In [7]:
from sdg_hub.core.blocks.llm import PromptBuilderBlock, LLMChatBlock, TextParserBlock

### Creating the required blocks

To get started, we'll construct the simplest possible flow for text classification using SDG Hub. We will focus on 3 main blocks that will often appear as a triplet while using `sdg_hub`

1. **Prompt Builder Block**: Converts each input text into a prompt formatted for the LLM. The important input argument to keep in mind for  `PromptBuilderblock` is the `prompt_config_path` which is where the prompt template is saved. Any prompt engineering we would want to do would be done in such a prompt template.
2. **LLM Chat Block**: Sends the prompt to the language model and receives its response (the predicted label).
3. **Text Parser Block**: Extracts the final label from the LLM's output.

This setup results in a single LLM interaction per sample, forming a minimal classification pipeline.

We are going to be using the simple prompt that can be found in `news_articles_classification_prompt.yaml`

In [8]:
promptbuilderblock_1 = PromptBuilderBlock(block_name='annotation_prompt_builder', input_cols=['text'], output_cols=['annotation_prompt'], prompt_config_path="news_classification_prompt.yaml", format_as_messages=True)
llmchatblock_1 = LLMChatBlock(block_name='annotation_llm_chat_block', input_cols=['annotation_prompt'], output_cols=['raw_output'], temperature=0.0, max_tokens=5, extra_body={'guided_choice': ['World', 'Sports', 'Business', 'Sci/Tech']}, async_mode=True)
textparserblock_1 = TextParserBlock(block_name='annotation_text_parser_block', input_cols=['raw_output'], output_cols=['output'], start_tags=[''], end_tags=[''])

### Designing the `Flow`

The `Flow` class is at the heart of SDG Hub. Simply put, a `Flow` is a chain of `Blocks` that get executed sequentially. Here, we will simply chain our PromptBuilder -> LLMChatBlock -> TextParser, in that order:

```mermaid
flowchart LR
    subgraph Flow
        direction LR
        A[PromptBuilderBlock] --> B[LLMChatBlock] --> C[TextParserBlock]
    end
```



In [9]:
flow = Flow(blocks=[promptbuilderblock_1, llmchatblock_1, textparserblock_1], metadata=FlowMetadata(name="annotation_flow", description="A flow for news article classification", author="sdg_hub"))

### Set the model configs for the `Flow`

In SDG Hub, model details such as the API base URL, the API Key (if any) and the model name are set at a Flow level using the `set_model_config` method as shown. The `model` parameter accepts a string in the format of "`provider`/`model_name`". Here our `provider` is 'hosted_vllm' as we are using a self hosted model through vllm, and the model name is "meta-llama/Llama-3.3-70B-Instruct"

In [ ]:
flow.set_model_config(model="hosted_vllm/meta-llama/Llama-3.3-70B-Instruct", api_base="http://localhost:8000/v1", api_key="")

### Time to generate!

In sdg_hub, the way to generate data is very simple. we simply use the `generate` method from `Flow`. At its simplest form, all the `generate` method needs is the input dataset to operate on. Additionally, we can pass runtime parameters for each block as well, if we wish to override any of the block specific model configs.

In [ ]:
generated_data = flow.generate(test_data)

### Evaluation

Now that we’ve generated synthetic labels using our simple classification flow, it’s time to evaluate how well the model performed. The goal of this section is to compare the predicted labels against the **true labels** from the dataset using standard classification metrics (precision, recall, f-1 score and classification accuracy)

We’ll use `sklearn.metrics.classification_report`, which provides precision, recall, F1-score, and support for each class.


In [12]:
print(classification_report(generated_data["category"], generated_data["output"]))

precision    recall  f1-score   support

    Business       0.47      1.00      0.64        32
    Sci/Tech       1.00      0.05      0.10        19
      Sports       0.93      1.00      0.96        27
       World       1.00      0.09      0.17        22

    accuracy                           0.62       100
   macro avg       0.85      0.54      0.47       100
weighted avg       0.81      0.62      0.52       100

## Introducing an Assessment step

Our initial flow used a one step approach — the model was given the task, a fixed label set, and some input text. While this baseline gives us a useful starting point, it has clear limitations:

- The model may rely on generic heuristics or surface patterns that don’t generalize well.
- It can confuse similar categories (e.g., "World" vs. "Business") without knowing how they're typically used.
- Without guidance, the model may underperform on edge cases or ambiguous queries.


### What is Assessment

With an assessment step, we will call to the same LLM, but this time, we provide the LLM with its own previous categorization label, and the original text. We will prompt the LLM to think about the original prediction, and give it context about challening cases
In this manner, we can elicit critical judgement from the model about its own prior classification decision. This type of additional context can be useful in the next iteration.


### What We’ll Do Next

We’ll now enhance our flow by introducing another chain of `PromptBuilder` -> `LLMChatBlock` -> `TextParserBlock` whose purpose is to pass the (original text +  prediction) to the LLM and obtain a verification or assessment of the prediction.


```mermaid
flowchart LR
 subgraph Flow1[Initial Classification]
 direction LR
 A[PromptBuilderBlock] --> B[LLMChatBlock] --> C[TextParserBlock]
 end
 subgraph Flow2[Assessment]
 direction LR
 D[PromptBuilderBlock_Assessment] --> E[LLMChatBlock_Assessment] --> F[TextParserBlock_Assessment]
 end
 
 C --> D
```


We will investigate if this catches any of the mis-classifications, and get an idea of how well our verification prompting works!

In [ ]:
promptbuilderblock_assessment = PromptBuilderBlock(block_name='verifier_prompt_builder', input_cols=['text', 'output'], output_cols=['assessment_prompt'], prompt_config_path="news_classification_assessment_prompt.yaml", format_as_messages=True)
llmchatblock_assessment = LLMChatBlock(block_name='verifier_llm_chat_block', input_cols=['assessment_prompt'], output_cols=['raw_assessment_output'], async_mode=True)
textparserblock_assessment = TextParserBlock(block_name='verifier_text_parser_block', input_cols=['raw_assessment_output'], output_cols=['assessment_output'], start_tags=[''], end_tags=[''])

flow = Flow(blocks=[promptbuilderblock_1, llmchatblock_1, textparserblock_1, promptbuilderblock_assessment, llmchatblock_assessment, textparserblock_assessment], metadata=FlowMetadata(name="annotation_flow", description="A flow for news article classification", author="sdg_hub"))
flow.set_model_config(model="hosted_vllm/meta-llama/Llama-3.3-70B-Instruct", api_base="http://localhost:8000/v1", api_key="")

generated_data = flow.generate(test_data)

In [16]:
generated_data_pd = generated_data.to_pandas()
mislabeled_samples = generated_data_pd[generated_data_pd["category"] != generated_data_pd["output"]]

print(Panel(mislabeled_samples.iloc[0]['assessment_output'], title="Assessment"))
print(Panel(str(mislabeled_samples.iloc[0]['category']), title="Ground truth label"))

╭────────────────────────────────────────────────── Assessment ───────────────────────────────────────────────────╮
│ Assessment: The predicted category of "Business" for this news article seems incorrect. The article is a review │
│ of a video game, "Half-Life 2", and discusses its technical aspects, graphics, and science fiction story. This  │
│ content aligns more closely with the Sci/Tech category, which includes tech news, technology innovations, and   │
│ product launches.                                                                                               │
│                                                                                                                 │
│ The mention of Valve Corp., a video game development company, and the discussion of the game's technical        │
│ features, such as graphics and original science fiction story, are strong indicators that this article belongs  │
│ to the Sci/Tech category. The article does not discuss corporate earnings, market movements, or general         │
│ economic indicators, which are typical topics of the Business category.                                         │
│                                                                                                                 │
│ Therefore, this could be a case of misclassification, and the correct category for this news article should be  │
│ Sci/Tech.                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Ground truth label ───────────────────────────────────────────────╮
│ Sci/Tech                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Great! Now we can see that the assessment step is working good, especially on the misclassified samples as shown above. The above is a hard example which has slipped past our original classification flow, but was caught by our assessment step's critical judgement.

### Revising the Classifications

We will now create our final revision step, which will take the results of the initial prediction and the assessment steps and pass it onto the LLM once again for a revised attempt at classifying the same input text. The flow can be imagined like so:

```mermaid
flowchart LR
 subgraph Flow1[Initial Classification]
 direction LR
 A[PromptBuilderBlock] --> B[LLMChatBlock] --> C[TextParserBlock]
 end
 subgraph Flow2[Assessment]
 direction LR
 D[PromptBuilderBlock_Assessment] --> E[LLMChatBlock_Assessment] --> F[TextParserBlock_Assessment]
 end
 subgraph Flow3[Revised Classification]
 direction LR
 G[PromptBuilderBlock_Revision] --> H[LLMChatBlock_Revision] --> I[TextParserBlock_Revision]
 end
 
 C --> D
 F --> G
```

In [ ]:
promptbuilderblock_revision = PromptBuilderBlock(block_name='revised_prompt_builder', input_cols=['text', 'output', 'assessment_output'], output_cols=['revised_prompt'], prompt_config_path="revise_news_classification_prompt.yaml", format_as_messages=True)
llmchatblock_revision = LLMChatBlock(block_name='revised_llm_chat_block', input_cols=['revised_prompt'], output_cols=['raw_revised_output'], temperature=0.0, max_tokens=5, extra_body={'guided_choice': ['World', 'Sports', 'Business', 'Sci/Tech']}, async_mode=True)
textparserblock_revision = TextParserBlock(block_name='revised_text_parser_block', input_cols=['raw_revised_output'], output_cols=['revised_output'], start_tags=[''], end_tags=[''])

flow = Flow(blocks=[promptbuilderblock_1, llmchatblock_1, textparserblock_1, promptbuilderblock_assessment, llmchatblock_assessment, textparserblock_assessment, promptbuilderblock_revision, llmchatblock_revision, textparserblock_revision], metadata=FlowMetadata(name="annotation_flow", description="A flow for news article classification", author="sdg_hub"))
flow.set_model_config(model="hosted_vllm/meta-llama/Llama-3.3-70B-Instruct", api_base="http://localhost:8000/v1", api_key="")

In [ ]:
generated_data = flow.generate(test_data)

In [19]:
print(classification_report(generated_data["category"], generated_data["revised_output"]))

precision    recall  f1-score   support

    Business       0.87      0.84      0.86        32
    Sci/Tech       0.73      0.84      0.78        19
      Sports       0.93      1.00      0.96        27
       World       0.94      0.77      0.85        22

    accuracy                           0.87       100
   macro avg       0.87      0.86      0.86       100
weighted avg       0.88      0.87      0.87       100

🔥 We improved the results drastically! Let us take a look at the number of mislabeled samples before and after the assessment + revision steps


In [20]:
generated_data_pd = generated_data.to_pandas()
num_mislabeled_output = (generated_data_pd["category"] != generated_data_pd["output"]).sum()
num_mislabeled_revised = (generated_data_pd["category"] != generated_data_pd["revised_output"]).sum()
print(f"Number of mislabeled samples (original output): {num_mislabeled_output}")
print(f"Number of mislabeled samples (revised output): {num_mislabeled_revised}")


Number of mislabeled samples (original output): 38

Number of mislabeled samples (revised output): 13

Great, we whave now improved the classification accuracy of our system by augmenting our naive classification flow by adding an assessment followed by a revision step


## ✅ Summary: What You’ve Learned

In this tutorial, you learned how to create your own flow for a custom use-case using `sdg_hub`, using the fundamental components: `Flow` and `Block`. You also learned how to create and structure the prompts. You learned how to design an assessment or a judgement step in order to improve the performance of the overall system. You started from scratch and evolved it into a robust, high-accuracy system.

## 🚀 What’s Next?

* Prompt Engineer! -  You can add examples for classifications directly in the classification steps and see how this improves the performance. In-context examples are extremely effective at aligning the model's outputs to the task at hand
* Try it out on your own data!